# Recognize eaf
Принимает eaf c пустыми спанами, распознаёт их и записывает в eaf

In [94]:
import re
import os
from tqdm import tqdm
from collections import Counter, OrderedDict
from bs4 import BeautifulSoup
import warnings
from bs4 import XMLParsedAsHTMLWarning
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning, module='bs4')
from pydub import AudioSegment
import librosa
import torch
from IPython.display import Audio

In [20]:
from transformers import AutoModelForCTC, Wav2Vec2BertProcessor

model = AutoModelForCTC.from_pretrained("tbkazakova/wav2vec-bert-2.0-even-pakendorf")
processor = Wav2Vec2BertProcessor.from_pretrained("tbkazakova/wav2vec-bert-2.0-even-pakendorf")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
filename = 'eve_KEP_20230808_biography_toasr.eaf'

In [7]:
# Получаем таймкоды границ
def get_timecodes(soup):
    timecodes = soup.find_all('time_slot')
    ts_msec = {}
    for timecode in timecodes:
        if timecode.get('time_value'):
            ts_msec[timecode.get('time_slot_id')] = int(timecode.get('time_value'))
    return ts_msec

In [84]:
id_text = []

wavname = filename[:-10]+'.wav'
audio, sr = librosa.load(wavname)


with open(filename) as f:
    file = f.read()
soup = BeautifulSoup(file)
ts_msec = get_timecodes(soup)

ann_ref = 0
ann_phrase = {}
for tier in soup.find_all('tier'):
    if tier.get('tier_id').startswith('tx'):
        anns = tier.find_all('alignable_annotation')
        for ann in tqdm(anns):
            start = int(ts_msec[ann.get('time_slot_ref1')]/1000*sr)
            end = int(ts_msec[ann.get('time_slot_ref2')]/1000*sr)
            
            segment = audio[start:end]
            librosa.resample(segment, orig_sr=sr, target_sr=16000)
            logits = model(torch.tensor(processor(segment,
                                                  sampling_rate=16000).input_features[0]).unsqueeze(0)).logits
          
            pred_ids = torch.argmax(logits, dim=-1)[0]
            text = processor.decode(pred_ids)
            id_text.append([ann.get('annotation_id'), text])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [16:45<00:00, 21.86s/it]


In [96]:
templ_past = '''(<ALIGNABLE_ANNOTATION ANNOTATION_ID="{ann_id}"(.|\n)*?)(<ANNOTATION_VALUE/>)'''
templ_furure = r'\1<ANNOTATION_VALUE>{text}</ANNOTATION_VALUE>'

In [101]:
new_eaf_filename = filename[:-10]+'_asr.eaf'
with open(filename, encoding='utf8') as f:
    new_eaf = f.read()

for ann_id, text in tqdm(id_text):
    past = templ_past.format(ann_id=ann_id)
    future = templ_furure.format(text=text)
    new_eaf = re.sub(past, future, new_eaf)
    
with open(new_eaf_filename, 'w', encoding='utf-8') as f:
    f.write(new_eaf)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 9156.57it/s]
